In [8]:
import numpy as np
import pandas as pd

In [9]:
rtdata2 = pd.read_csv('dataset2/rtdata.txt',' ',names=['User ID','Service ID','Time Slice ID','Response Time (sec)'])
tpdata2 = pd.read_csv('dataset2/tpdata.txt',' ',names=['User ID','Service ID','Time Slice ID','Throughput (kbps)'])

In [10]:
rtdata2.head()

,User ID,Service ID,Time Slice ID,Response Time (sec)
0,0,0,0,4.180
1,0,1,0,0.416
2,0,2,0,0.441
3,0,3,0,2.764
4,0,4,0,0.465


In [11]:
tpdata2.head()

,User ID,Service ID,Time Slice ID,Throughput (kbps)
0,0,0,0,0.086842
1,0,1,0,1.637019
2,0,2,0,1.902494
3,0,3,0,0.131331
4,0,4,0,0.780645


In [14]:
for row in rtdata2.iterrows():
    print(row)
    break

(0, User ID                0.00
Service ID             0.00
Time Slice ID          0.00
Response Time (sec)    4.18
Name: 0, dtype: float64)


In [36]:
x= rtdata2.groupby(['User ID','Service ID','Time Slice ID'])['Response Time (sec)'].apply(list).reset_index()

In [23]:
x.head()

['User ID', 'Service ID', 'Time Slice ID', 'Response Time (sec)']